This notebook iterate pickle files calculated from the cut-images script. it calculates the mean and std per plant per date per time if it has radiance below the white reference panel. 
The script takes the mean of the 80% highest albedo pixel per plant.
albedo is calculated as the mean across the entire spectral range 400-1000 nm.
There is an additional step which is commented out that can check across the spectrum for values above 1 and below 0 and exclude entire spectra which have such values. however this causes loss of many pixel since many of this variations are caused because of water absorption bands. 


In [ ]:
import numpy as np
import pandas as pd
import os


In [ ]:
"""
Works fine but slow to load df to memory - not any more
"""
path = r'D:\DONE'
file_list = files = [os.path.join(dirpath, f)
    for dirpath, dirnames, files in os.walk(path)
        for f in files if f.endswith('pickle')]
collector = {}
for file in file_list:
    print (file)
    df = pd.read_pickle(file) # read pickle file
    spectralon = df[df['name']=='spectralon'].iloc[:,5:] #get spectralon radiance
    df_rad = df[df.type=='rad'] #just radiance
    df_rad = df_rad[:-2].copy() #drop spectralon and black
    df_rad.reset_index(drop=True, inplace = True) #reset index to match reflectance
    print (df_rad['name'].unique())
    df_ref = df[df.type=='ref'] #just reflectance
    df_rad = df_rad[:-2].copy()
    df_ref.reset_index(drop=True, inplace = True) #reset index to match rad
    print (df_rad['name'].unique())
    a = df_rad.iloc[:,2].where(((df_rad.iloc[:,5:].values<spectralon.values).sum(axis=1))==448).index.values #select pixels which are under the spectralon
    df = df_ref.iloc[a].copy() #select the pixels for analysis
    df.name = df.name.cat.remove_unused_categories()
    print (df['name'].unique())
#     df = df[df.type=='ref'] #keep only reflectance
#     df.reset_index(drop=True, inplace = True) #reindex
    df.drop(columns = ['coord'],inplace=True) # don't need coord anymore
    df = df.drop_duplicates() #drop duplicates 
    df['albedo'] = df.iloc[:,5:].mean(axis=1) #make albedo column
    cols = df.columns.tolist() # columns list
    df = df[[cols[-1]] + cols[:-1]] # bring albedo column to first
    names_count = df.name.value_counts() #how many pixels per plant
    count_80_prcnt = np.round(0.8*names_count).astype(int).copy() #how many pixels is 80%
    mean_spec = {} #mean spectrum holder
    std_spec = {}
    idx_80_prcnt = {} #index of 80%
    for i,j in count_80_prcnt.iteritems(): # e.g 13D     4687
        print (i)
        idx_80_prcnt[i] = df['albedo'][df['name'] == i].sort_values(ascending=False).iloc[:j].index
#         a = df[df['name']==i].loc[idx_80_prcnt[i]].iloc[:,5:]
#         a.loc[((a.loc[:,:]>0) & (a.loc[:,:]<1)).sum(axis=1)==448] #0,1,448 strong threshold
        mean_spec[i] = df[df['name']==i].loc[idx_80_prcnt[i]].iloc[:,5:].mean(axis=0)
        std_spec[i] = df[df['name']==i].loc[idx_80_prcnt[i]].iloc[:,5:].std(axis=0)
    collector[file[-26:-7]] = [mean_spec, std_spec]     
winsound.PlaySound('sound.wav', winsound.SND_FILENAME)

    

In [ ]:
c=[]
for key in collector:
    a = pd.DataFrame(collector[key][0]).T
    b = pd.DataFrame(collector[key][1]).T
    a['time'] = ['2018-09-19_07-05-57' for i in range(a.shape[0])]
    a['type'] = 'MEAN'
    b['type'] = 'STD'
    b['time'] = ['2018-09-19_07-05-57' for i in range(a.shape[0])]
    a.time = pd.to_datetime(a.time,format='%Y-%m-%d_%H-%M-%S')
    b.time = pd.to_datetime(a.time,format='%Y-%m-%d_%H-%M-%S')
    a = a.set_index(['time','type'], append=True)
    b = b.set_index(['time','type'], append=True)
    c.append(a.append(b))
df = pd.concat(c)    

In [ ]:
df.to_pickle(r'D:\DONE\combined')